In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv

## Load the file as npz for size reasons

In [2]:
natality = np.load('mat2004_data_file.npz', allow_pickle=True)

In [5]:
print(natality.files)

us_natality2004 = natality['X']

['X']


In [6]:
# Display the first few rows of the data frame
print(type(us_natality2004))
print(us_natality2004[:5])

<class 'numpy.ndarray'>
[['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']
 ['S' '1' '2004' ... '' '' '']]


In [7]:
print(us_natality2004.shape)
print(us_natality2004[1])

(4118907, 337)
['S' '1' '2004' '1' '7' 'AK' 'AK' '999' '9' '' '2' '2' '' '' '34' '10' '5'
 '' 'AK' '1' 'AK' 'AK' '999' '99999' '0' '9999' '9' '9' '9' '2' '1' '1' ''
 '1' '1' '' '0' '6' '1' '' '' '16' '5' '' '99' '' '33' '5' '' '' '1' '0'
 '6' '1' '0' '0' '0' '1' '1' '1' '1' '' '' '' '' '2' '1' '1' '10' '6' ''
 '42' '7' '3' '' '' '' '' '2' '0' '0' '' '2' '0' '0' '' '' '' '' '' '' ''
 '' '' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '' '' '' '' '2' '2' '2' '2' '2' '1' '2' '' '' '' '' '' '' '' '' '' '' ''
 '' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '' ''
 '' '' '' '1' '2' '2' '2' '2' '2' '1' '4' '9' '4' '1' '' 'M' '' '4' '4'
 '2003' '39' '39' '6' '2' '' '' '3289' '7' '3' '' '' '' '' '' '' '' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '' '' '' '' '' '' '' '' '' '' '' '' '2'
 '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2' '2'
 '2' '2' '2' '1' '1' '0' '1' '1' '0' '1' '1' '1' '0' '1' '0' '1' '1' '1'
 '1' '1' '1' '1' '1' '1' '1' 

In [10]:
print(us_natality2004[:,173])

['40' '39' '41' ... '41' '38' '99']


In [11]:
# OCNTYFIPS is column index 7 which identifies the county in the state
# column index 7 gives county code within state
# column index 5 goves the state where the birth occurred - OSTATE
# to get full identifier, need state and county code

print(us_natality2004[:,7])

['20' '999' '20' ... '47' '81' '81']


## Code to associate a county name to a state abbreviation and county code

In [63]:
state_col = us_natality2004[:, 5]  # column with state abbreviation
county_col = us_natality2004[:, 7]  # column with county code

In [64]:
# Make the county codes zero-padded strings
county_codes = np.char.zfill(county_col.astype(str), 3)
state_abbrs = np.char.upper(state_col.astype(str))

KeyboardInterrupt: 

In [ ]:
# Load full county FIPS-to-name mapping
county_df = pd.read_csv(
    "https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv",
    dtype={"state_fips": str, "county_fips": str, "fips": str},
    encoding="iso-8859-1"
)

In [ ]:
# Load the FIPS-to-county-name map
fips_df = pd.read_csv(
    "https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv",
    #dtype={'state': str, 'county_fips': str},
    #usecols=['state', 'county_fips', 'county_name'],
    dtype={'state': str, 'fips': str, 'county': str},
    usecols=['state', 'fips', 'county'],  
    encoding="iso-8859-1"
)

fips_df = fips_df.rename(columns={'fips': 'county_fips', 'county': 'county_name'})

In [ ]:
# make a dictionary for (state_abbr, county_code) -> county_name
county_dict = {
    (row.state.upper(), row.county_fips.zfill(3)): row.county_name
    for row in county_df.itertuples()
}


In [ ]:
# Vectorized lookup using list
county_names = np.array([
    county_dict.get((state, county), 'Unknown County')
    for state, county in zip(state_abbrs, county_codes)
])

In [ ]:
#new dataset that includes the county name 
us_natality_with_county = np.column_stack((us_natality2004, county_names))


## Air Quality Dataset

In [12]:
air_quality_by_county = pd.read_csv('annual_aqi_by_county_2004.csv')

print(air_quality_by_county.head())

     State   County  Year  Days with AQI  Good Days  Moderate Days  \
0  Alabama  Baldwin  2004            271        158             98   
1  Alabama  Barbour  2004             49         11             37   
2  Alabama     Clay  2004            283        167            112   
3  Alabama  Colbert  2004            276        178             98   
4  Alabama   DeKalb  2004            361        228            126   

   Unhealthy for Sensitive Groups Days  Unhealthy Days  Very Unhealthy Days  \
0                                   15               0                    0   
1                                    1               0                    0   
2                                    4               0                    0   
3                                    0               0                    0   
4                                    7               0                    0   

   Hazardous Days  Max AQI  90th Percentile AQI  Median AQI  Days CO  \
0               0      143      

In [13]:
# Figuring out how big these files are 
print(us_natality2004.shape)
print(air_quality_by_county.shape)

(4118907, 337)
(1121, 18)


Some useful column codes for health conditions of interest:

2004 Important codes:

- URF_ANEMIA: Anemia 
- URF_PHYPER: Pregnancy Associated Hypertension
- URF_ECLAM: Eclampsia
- GESTREC3: Combined Gestation Recode 3 (preterm or not)
- ESTGEST: Obstetric/Clinical Gestation Est (should be removed if predicting preterm)
- DBWT: Birth Weight – Detail in Grams (should remove if predicting low birth weight)
- BWTR4: Birth Weight Recode 4 (convenient categories)

In [14]:
exclude_cols = ['revision', 'recwt', 'dob_yy', 'xostate', 'bfacil', 'mage_impflg', 'mage_repflg', 'mbcntry', 'xmrstate', 'restatus', 
                'mbrace', 'mracerec', 'mraceimp', 'umhisp', 'mar_imp', 'meduc', 'fagerpt_flg', 'fagecomb', 'fagerec11', 'fbrace', 
                'fraceimp', 'ufhisp', 'lbo_rec', 'tbo_rec', 'dllb_mm', 'dllb_yy', 'precare', 'precare_rec',  'mpcb_rec6', 'mpcb_rec5',
                'previs_rec', 'apncu', 'wtgain_rec', 'dfpc_imp', 'cig_1', 'cig_2', 'cig_3',  'cig_rec6', 'cig_rec', 'alcohol', 'drinks_rec', 
                'rf_diab', 'rf_gest', 'rf_phyp','rf_ghyp','rf_eclam', 'rf_ppterm', 'rf_ppoutc','rf_cesar', 'rf_cesarn',
                'op_cerv', 'op_tocol', 'op_ecvs', 'op_ecvf',  'on_ruptr', 'on_precip', 'on_prol', 'ld_indl', 'ld_augm', 'ld_nvpr',
                'ld_ster', 'ld_anti', 'ld_chor', 'ld_mecs', 'ld_fint', 'ld_anes',  'me_attf', 'me_attv', 'me_pres', 'me_rout', 'me_trial',
                'ume_vag', 'ume_vbac', 'ume_primc', 'ume_repec', 'ume_forcep', 'ume_vac', 'apgar5r', 'imp_plur', 'imp_sex',  'dlmp_mm',
                'dlmp_dd', 'dlmp_yy', 'combgest', 'gestrec10',  'obgest_flg', 'gest_imp', 'bwtr12',  'ab_aven1', 'ab_aven6', 'ab_nicu',
                'ab_surf', 'ab_anti', 'ab_seiz', 'ab_binj', 'ca_anen', 'ca_mnsb', 'ca_cchd', 'ca_cdh', 'ca_omph', 'ca_gast', 'ca_limb',
                'ca_cleft', 'ca_clpal', 'ca_down', 'ca_disor', 'ca_hypo',  'f_morigin', 'f_forigin', 'f_meduc', 'f_clinest', 'f_apgar5',
                'f_tobaco', 'f_med', 'f_wtgain', 'f_alcol', 'f_api','f_tobac','f_mpcb','f_mpcb_u', 'f_urf_anemia', 'f_urf_cardiac',
                'f_urf_lung', 'f_urf_diabetes', 'f_urf_herpes', 'f_urf_hydra', 'f_urf_hemo', 'f_urf_chyper', 'f_urf_phyper', 'f_urf_eclamp',
                'f_urf_incervix', 'f_urf_pre4000', 'f_urf_preterm', 'f_urf_renal', 'f_urf_rh', 'f_urf_uterine', 'f_urf_othermr', 
                'f_uob_amnio', 'f_uob_monitor', 'f_uob_induct', 'f_uob_stimul', 'f_uob_tocol', 'f_uob_ultras', 'f_uob_otherob', 
                'f_uld_febrile', 'f_uld_meconium', 'f_uld_rupture', 'f_uld_abruptio', 'f_uld_preplace', 'f_uld_excebld', 'f_uld_seizure',
                'f_uld_precip', 'f_uld_prolg', 'f_uld_dysfunc', 'f_uld_breech', 'f_uld_cephalo', 'f_uld_cord', 'f_uld_anest', 'f_uld_distr',
                'f_uld_other', 'f_u_vag', 'f_u_vbac', 'f_u_primc', 'f_u_repec', 'f_u_forcep', 'f_u_vac', 'f_uab_anemia', 'f_uab_injury',
                'f_uab_alcosyn', 'f_uab_hyaline', 'f_uab_meconsyn', 'f_uab_venl30', 'f_uab_ven30m', 'f_uab_seiz', 'f_uab_otherab', 
                'f_uca_anen', 'f_uca_spina', 'f_uca_hydro',  'f_uca_microce', 'f_uca_nervous', 'f_uca_heart', 'f_uca_circul', 'f_uca_rectal',
                'f_uca_tracheo', 'f_uca_omphalo', 'f_uca_gastro', 'f_uca_genital', 'f_uca_renalag', 'f_uca_urogen', 'f_uca_cleftlp',
                'f_uca_adactyl', 'f_uca_club', 'f_uca_hernia', 'f_uca_musculo', 'f_uca_downs', 'f_uca_chromo', 'f_uca_othrcon']

#include_cols = []

col_names = ['revision',
 'recwt',
 'dob_yy',
 'dob_mm',
 'dob_wk', # eh not sure
 'ostate',
 'xostate', # this makes NYC its own state - how do we feel - i took out
 'ocntyfips',
 'ocntypop',
 'bfacil', #10
 'ubfacil',
 'bfacil3',
 'mage_impflg', # blank 
 'mage_repflg', # blank
 'mager',
 'mager14',
 'mager9',
 'mbcntry', # blank, 18
 'umbstate',
 'mbstate_rec',
 'xmrstate', # agin making NYC own state - i think take out
 'mrstate',
 'mrcntyfips',
 'mrcityfips',
 'cmsa',
 'msa',
 'msa_pop',
 'rcnty_pop',
 'rcity_pop',
 'metrores',
 'rectype',
 'restatus',
 'mbrace', # blank , 33
 'mrace',
 'mracerec',
 'mraceimp', #36 blank
 'umhisp',
 'mracehisp',
 'mar',
 'mar_imp', #40 blank
 'meduc', # 41 blank
 'dmeduc',
 'meduc_rec',
 'fagerpt_flg', # 44 blank
 'fagerpt',
 'fagecomb', #46 blank
 'ufagecomb',
 'fagerec11',
 'fbrace', #49 blank
 'fraceimp', # 50 blank
 'fracerec',
 'ufhisp',
 'fracehisp',
 'frace',
 'priorlive',
 'priordead',
 'priorterm',
 'lbo',
 'lbo_rec',
 'tbo',
 'tbo_rec',
 'dllb_mm', # 62 thru
 'dllb_yy',
 'precare',
 'precare_rec', # 65
 'mpcb',
 'mpcb_rec6',
 'mpcb_rec5',
 'uprevis',
 'previs_rec',
 'apncu', # 71
 'wtgain',
 'wtgain_rec',
 'u_apncu',
 'dfpc_imp', #75 thru
 'cig_1',
 'cig_2',
 'cig_3', # 78
 'tobuse',
 'cigs',
 'cig_rec6',
 'cig_rec', #82
 'alcohol',
 'drinks',
 'drinks_rec',
 'rf_diab', #86 thru
 'rf_gest',
 'rf_phyp',
 'rf_ghyp',
 'rf_eclam',
 'rf_ppterm',
 'rf_ppoutc',
 'rf_cesar',
 'rf_cesarn', # end94
 'urf_anemia',
 'urf_card',
 'urf_lung',
 'urf_diab',
 'urf_gen',
 'urf_hydra',
 'urf_hemo',
 'urf_chyper',
 'urf_phyper',
 'urf_eclam',
 'urf_incerv',
 'urf_pre4000',
 'urf_preterm',
 'urf_renal',
 'urf_rh',
 'urf_uterine',
 'urf_other',
 'op_cerv', #112 thur
 'op_tocol',
 'op_ecvs',
 'op_ecvf', #115
 'uop_amnio',
 'uop_monit',
 'uop_induc',
 'uop_stiml',
 'uop_tocol',
 'uop_ultra',
 'uop_other',
 'on_ruptr', #123 thru
 'on_precip',
 'on_prol',
 'ld_indl',
 'ld_augm',
 'ld_nvpr',
 'ld_ster',
 'ld_anti',
 'ld_chor',
 'ld_mecs',
 'ld_fint',
 'ld_anes', #134
 'uld_febr',
 'uld_meco',
 'uld_ruptr',
 'uld_abrup',
 'uld_prepla',
 'uld_excbl',
 'uld_seiz',
 'uld_precip',
 'uld_prolg',
 'uld_dysfn',
 'uld_breech',
 'uld_cephal',
 'uld_cord',
 'uld_anest',
 'uld_distr',
 'uld_other',
 'me_attf', #151 thru
 'me_attv',
 'me_pres',
 'me_rout',
 'me_trial', #155
 'ume_vag',
 'ume_vbac',
 'ume_primc',
 'ume_repec',
 'ume_forcep',
 'ume_vac',
 'dmeth_rec',
 'attend',
 'apgar5',
 'apgar5r',
 'dplural',
 'imp_plur', #167
 'sex',
 'imp_sex', #169
 'dlmp_mm',
 'dlmp_dd',
 'dlmp_yy',
 'estgest',
 'combgest',
 'gestrec10',
 'gestrec3',
 'obgest_flg', #177
 'gest_imp', #178
 'dbwt',
 'bwtr12',
 'bwtr4',
 'ab_aven1', #182 thru
 'ab_aven6',
 'ab_nicu',
 'ab_surf',
 'ab_anti',
 'ab_seiz',
 'ab_binj', #188
 'uab_anem',
 'uab_injury',
 'uab_alcoh',
 'uab_hyal',
 'uab_mecon',
 'uab_venl30',
 'uab_ven30m',
 'uab_nseiz',
 'uab_other',
 'ca_anen', #198 to 209
 'ca_mnsb',
 'ca_cchd',
 'ca_cdh',
 'ca_omph',
 'ca_gast',
 'ca_limb',
 'ca_cleft',
 'ca_clpal',
 'ca_down',
 'ca_disor',
 'ca_hypo', #209
 'uca_anen', # from here to next marker I could be persuaded to take out
 'uca_spina',
 'uca_hydro',
 'uca_micro',
 'uca_nerv',
 'uca_heart',
 'uca_circ',
 'uca_rectal',
 'uca_trach',
 'uca_ompha',
 'uca_gastro',
 'uca_genital',
 'uca_renal',
 'uca_urogen',
 'uca_cleftlp',
 'uca_adactyly',
 'uca_clubft',
 'uca_hernia',
 'uca_muscu',
 'uca_downs',
 'uca_chromo',
 'uca_other', # end of newborn congenital anomalies
 'f_morigin', # start of flag reporting
 'f_forigin',
 'f_meduc',
 'f_clinest',
 'f_apgar5',
 'f_tobaco',
 'f_med',
 'f_wtgain',
 'f_alcol',
 'f_api',
 'f_tobac',
 'f_mpcb',
 'f_mpcb_u',
 'f_urf_anemia',
 'f_urf_cardiac',
 'f_urf_lung',
 'f_urf_diabetes',
 'f_urf_herpes',
 'f_urf_hydra',
 'f_urf_hemo',
 'f_urf_chyper',
 'f_urf_phyper',
 'f_urf_eclamp',
 'f_urf_incervix',
 'f_urf_pre4000',
 'f_urf_preterm',
 'f_urf_renal',
 'f_urf_rh',
 'f_urf_uterine',
 'f_urf_othermr',
 'f_uob_amnio',
 'f_uob_monitor',
 'f_uob_induct',
 'f_uob_stimul',
 'f_uob_tocol',
 'f_uob_ultras',
 'f_uob_otherob',
 'f_uld_febrile',
 'f_uld_meconium',
 'f_uld_rupture',
 'f_uld_abruptio',
 'f_uld_preplace',
 'f_uld_excebld',
 'f_uld_seizure',
 'f_uld_precip',
 'f_uld_prolg',
 'f_uld_dysfunc',
 'f_uld_breech',
 'f_uld_cephalo',
 'f_uld_cord',
 'f_uld_anest',
 'f_uld_distr',
 'f_uld_other',
 'f_u_vag',
 'f_u_vbac',
 'f_u_primc',
 'f_u_repec',
 'f_u_forcep',
 'f_u_vac',
 'f_uab_anemia',
 'f_uab_injury',
 'f_uab_alcosyn',
 'f_uab_hyaline',
 'f_uab_meconsyn',
 'f_uab_venl30',
 'f_uab_ven30m',
 'f_uab_seiz',
 'f_uab_otherab',
 'f_uca_anen',
 'f_uca_spina',
 'f_uca_hydro',
 'f_uca_microce',
 'f_uca_nervous',
 'f_uca_heart',
 'f_uca_circul',
 'f_uca_rectal',
 'f_uca_tracheo',
 'f_uca_omphalo',
 'f_uca_gastro',
 'f_uca_genital',
 'f_uca_renalag',
 'f_uca_urogen',
 'f_uca_cleftlp',
 'f_uca_adactyl',
 'f_uca_club',
 'f_uca_hernia',
 'f_uca_musculo',
 'f_uca_downs',
 'f_uca_chromo',
 'f_uca_othrcon', #end flag reporting
 'mrace1e', # here down blank (final 16)
 'mrace2e',
 'mrace3e',
 'mrace4e',
 'mrace5e',
 'mrace6e',
 'mrace7e',
 'mrace8e',
 'frace1e',
 'frace2e',
 'frace3e',
 'frace4e',
 'frace5e',
 'frace6e',
 'frace7e',
 'frace8e']

### To Do

- exclude those columns listed
- try again to associate county names to codes, perhaps on the AQI dataset
- switch columns to integer format


### Get rid of those columns

In [ ]:
# Get indices of columns to keep
cols_to_keep = [i for i, name in enumerate(col_names) if name not in exclude_cols]

# Filter the array
filtered_natality = us_natality2004[:, cols_to_keep]

# Optionally, get the new column names too
filtered_col_names = [col_names[i] for i in cols_to_keep]

In [ ]:
print(filtered_natality[1])